터미널에서   
conda install -c anaconda requests  https://anaconda.org/anaconda/requests  
conda install bs4   
conda install lxml

In [ ]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
import re #정규식 지원

In [ ]:
url='https://product.kyobobook.co.kr/detail/S000001619177'
r = requests.get(url)
print(r)
html = requests.get(url).text
soup = BeautifulSoup(html, 'lxml')
print(soup)

In [ ]:
def extract_number(s):
    return ''.join(filter(str.isdigit, s)) #문자열을 입력받아 숫자만 추출하여 문자열로 반환

In [ ]:
url='https://product.kyobobook.co.kr/detail/S000001619177'
html = requests.get(url).text
soup = BeautifulSoup(html, 'lxml')
title_tag = 'div.auto_overflow_inner>h1>span.prod_title'
title = soup.select(title_tag)
print(title)
author_tag = 'div.author> a'
author = soup.select(author_tag)
print(author)
publisher_tag = 'div.prod_info_text.publish_date > a'
publisher = soup.select(publisher_tag)
print(publisher)
price_tag = 'li.prod_type_item.active > a > span.prod_price'
price_origin = soup.select_one(price_tag).text
price = extract_number(price_origin)
print(price)
isbn_tag = '#scrollSpyProdInfo > div.product_detail_area.basic_info > div.tbl_row_wrap > table > tbody > tr:nth-child(1) > td'
isbn_origin = soup.select_one(isbn_tag).text
isbn_pattern = r'\b978[\d]{10}\b'
isbn_list = re.findall(isbn_pattern, isbn_origin)
isbn = isbn_list[0]
print(isbn)
img_tag = 'div.portrait_img_box > img'
img_url = soup.select(img_tag)[0].get('src')
print(img_url)

In [ ]:
print(len(title))
print('title '+title[0].text)

print(len(author))
print('author '+author[0].text)

print(len(publisher))
print('publisher '+publisher[0].text)

print(len(price))
print('price '+price)

print(len(isbn))
print('isbn '+isbn)

print(len(img_url))
print('img_url '+img_url)

In [ ]:
# def extract_number(s):
#     return ''.join(filter(str.isdigit, s)) #문자열을 입력받아 숫자만 추출하여 문자열로 반환

def extract_number(s):
    return int(''.join(filter(str.isdigit, s))) #int() 함수로 변환

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from urllib.request import urlopen


driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

url_1 = "https://product.kyobobook.co.kr/category/KOR/010303#?page="
url_2 = "&type=best&per=50"

# 교보문고의 베스트셀러 웹페이지를 가져옵니다.
driver.get("https://product.kyobobook.co.kr/bestseller/online?period=003#?page=1&per=50&ymw=&period=003&saleCmdtClstCode=01&dsplDvsnCode=001&dsplTrgtDvsnCode=004&saleCmdtDsplDvsnCode=")
bsObject = BeautifulSoup(driver.page_source, 'html.parser')
# print(bsObject)

# 책의 상세 웹페이지 주소를 추출하여 리스트에 저장합니다.
book_page_urls = []
for item in bsObject.find_all('div', 'prod_info_box'):
    url = item.select('a')[0].get('href')
    book_page_urls.append(url)
    
    
print(book_page_urls)
print(len(book_page_urls))

book_list=[]

# 웹페이지로부터 필요한 정보를 추출합니다. 
for index, book_page_url in enumerate(book_page_urls):
    html = requests.get(book_page_url).text
    soup = BeautifulSoup(html, 'lxml')
    title_tag = 'div.auto_overflow_inner>h1>span.prod_title'
    title = soup.select(title_tag)[0].text
#     print(title)
    author_tag = 'div.author> a'
    author = soup.select(author_tag)[0].text
#     print(author)
    publisher_tag = 'div.prod_info_text.publish_date > a'
    publisher = soup.select(publisher_tag)[0].text
#     print(publisher)
    price_tag = 'span.sale_price > s.val'
    price_origin = soup.select_one(price_tag).text
#     price = extract_number(price_origin) #함수사용
    price = int(re.sub('[^0-9]', '', price_origin)) #정규표현식사용
#     print(price)
    isbn_tag = '#scrollSpyProdInfo > div.product_detail_area.basic_info > div.tbl_row_wrap > table > tbody > tr:nth-child(1) > td'
    isbn_origin = soup.select_one(isbn_tag).text
    isbn = isbn_origin[:13]
#     print(isbn)
    img_tag = 'div.portrait_img_box > img'
    img_url = soup.select(img_tag)[0].get('src')
#     print(img_url)

#     print(title, author, publisher, price, isbn, img_url)

    book_list.append([title, author, publisher, price, isbn, img_url, "소설"])
    
print(book_list)
# colList = ['title', 'author', 'publisher', 'price', 'isbn', 'img_url', 'category']
# df = pd.DataFrame(np.array(book_list), columns=colList)
# df.to_csv("E:\source\scraping/교보.csv", index = False)

In [ ]:
colList = ['title', 'author', 'publisher', 'price', 'isbn', 'img_url', 'category']
df = pd.DataFrame(np.array(book_list), columns=colList)
df.to_csv("E:\source\scraping/교보베스트.csv", index = False)

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from urllib.request import urlopen


driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

url_1 = "https://product.kyobobook.co.kr/bestseller/online?period=003#?page="
url_2 = "&per=50&ymw=&period=003&saleCmdtClstCode=01&dsplDvsnCode=001&dsplTrgtDvsnCode=004&saleCmdtDsplDvsnCode="


for i in range (1,11):
    
    url = url_1+str(i)+url_2
    print('# Page', i)

    
    # 교보문고의 웹페이지를 가져옵니다.
    driver.get(url)
    bsObject = BeautifulSoup(driver.page_source, 'lxml')
#     print(bsObject)

    # 책의 상세 웹페이지 주소를 추출하여 리스트에 저장합니다.
    book_page_urls = []
    for item in bsObject.select('div.prod_info_box'):
        url = item.select('a')[0].get('href')
        book_page_urls.append(url)


    print(book_page_urls)
    print(len(book_page_urls))
    
    print('# Page', i)

    book_list=[]

    # 웹페이지로부터 필요한 정보를 추출합니다. 
    for index, book_page_url in enumerate(book_page_urls):
        html = requests.get(book_page_url).text
        soup = BeautifulSoup(html, 'lxml')
        title_tag = 'div.auto_overflow_inner>h1>span.prod_title'
        title = soup.select(title_tag)[0].text
    #     print(title)
        author_tag = 'div.author> a'
        author = soup.select(author_tag)[0].text
    #     print(author)
        publisher_tag = 'div.prod_info_text.publish_date > a'
        publisher = soup.select(publisher_tag)[0].text
    #     print(publisher)
        price_tag = 'span.sale_price > s.val'
        price_origin = soup.select_one(price_tag).text
    #     price = extract_number(price_origin) #함수사용
        price = int(re.sub('[^0-9]', '', price_origin)) #정규표현식사용
    #     print(price)
        isbn_tag = '#scrollSpyProdInfo > div.product_detail_area.basic_info > div.tbl_row_wrap > table > tbody > tr:nth-child(1) > td'
        isbn_origin = soup.select_one(isbn_tag).text
        isbn = isbn_origin[:13]
    #     print(isbn)
        img_tag = 'div.portrait_img_box > img'
        img_url = soup.select(img_tag)[0].get('src')
    #     print(img_url)

    #     print(title, author, publisher, price, isbn, img_url)

        book_list.append([title, author, publisher, price, isbn, img_url, "소설"])

    print(book_list)
    # colList = ['title', 'author', 'publisher', 'price', 'isbn', 'img_url', 'category']
    # df = pd.DataFrame(np.array(book_list), columns=colList)
    # df.to_csv("E:\source\scraping/교보.csv", index = False)

In [ ]:
colList = ['title', 'author', 'publisher', 'price', 'isbn', 'img_url', 'category']
df = pd.DataFrame(np.array(book_list), columns=colList)
df.to_csv("E:\source\scraping/교보333.csv", index = False)

In [ ]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [ ]:
import time

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 교보문고의 웹페이지를 가져옵니다.
driver.get("https://product.kyobobook.co.kr/bestseller/online?period=003#?page=1&per=50&ymw=&period=003&saleCmdtClstCode=01&dsplDvsnCode=001&dsplTrgtDvsnCode=004&saleCmdtDsplDvsnCode=")
bsObject = BeautifulSoup(driver.page_source, 'lxml')
# print(bsObject)

books = bsObject.select("ol.prod_list")
print(books)



# print(book_page_urls)
# print(len(book_page_urls))

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.set_page_load_timeout(30) # 30초로 설정

driver.get("https://product.kyobobook.co.kr/category/KOR/0103#?page=3&type=all&per=20&sort=new")

wait = WebDriverWait(driver, 10)
element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "li.list_item")))
bsObject = BeautifulSoup(driver.page_source, 'lxml')
print(bsObject)

books = bsObject.select("li.proud_item")
for book in books:
    title = book.select_one("div.title > a").text.strip()
    author = book.select_one("div.author").text.strip()
    price = book.select_one("div.price > strong").text.strip()
    print(title, author, price)